<a href="https://www.kaggle.com/code/tarakenny/fitbit-sample-vs-aha-guideline-analysis?scriptVersionId=229887555" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## FitBit sample vs AHA Guideline Analysis for BellaBeat Company
author: "Tara Kenny"
date: "2025-26-03"



## Introduction to BellaBeat and Analysis Opportunity
BellaBeat is a high-tech manufacturer of health-focused products for women. BellaBeat intends to become a larger player in the  global smart device market. 
Urška Sršen: BellaBeat’s cofounder and Chief Creative Officer has asked me to analyze smart device usage data in order to gain insight into how people are already using their smart devices. She would like a high-level recommendations for how these trends can inform BellaBeat marketing strategy.  

## Sample Overview
Ms. Sršen has asked me to focus on specific fitness tracker Data available on Kaggle - "FitBit Fitness Tracker Data" (CC0: Public Domain, dataset made available through Mobius).

The data is organized as a zip file dataset made up of 20 files, zipped into 2 folders: 

 * 	data collected from March 12, 2016 through April 11, 2016 and 
 * 	data collected from April 12, 2016 through May 12, 2016.
 
Most data is in long data format – each field in first column is repeated to show different times for each ID and characteristics collected for that field.  Each row represents either a day, minute or hour observation for each athlete, Each subject can be represented by multiple rows.

It's important to note this is a very small sample so not considered reliable.   It is comprised of data collected from 35 fitbit users over less than 2 months.  Worldwide fitbit user population size is 38M (Statista Fitbit monthly active users 2012-2023, Published by Federica Laricchia, Jun 7, 2024).  To achieve a target 95% confidence level with a 5% margin of error, we would need a sample size of 385 users.  As it stands, our sample size of 35 implies a margin of error of about 18% at 95% confidence level.  Margin of error is a tolerable degree of deviation from a correct or exact value or target.
The data also did not include sex, age, weight loss or muscle mass for each athlete. 

But what I found most valuable was data pertaining to minutes exercised at different intensity levels.   


## Targeted Deliverable

The most consistent data available across the samples were minutes at different intensity levels of exercise.  I leveraged this data to determine how well our sample is meeting ADA guidelines for weekly exercise overall.  
The American Heart Association(AHA) suggests getting at least 150 minutes per week of moderate-intensity aerobic activity or 75 minutes per week of vigourous aerobic activity.  
(https://www.heart.org/en/healthy-living/fitness/fitness-basics/aha-recs-for-physical-activity-in-adults)
(https://www.cdc.gov/physical-activity-basics/guidelines/adults.html#)

I also used the heartrate and METs information available in the data to compare to AHA targets for each at similar intensities (https://www.heart.org/en/healthy-living/fitness/fitness-basics/target-heart-rates)

I have made all the R code visible in this document to make the logic to the findings transparent.

## Data Processing - merging and cleanup of data

In [ ]:
install.packages("tidyverse", repos = "http://cran.us.r-project.org")
library(tidyverse)
install.packages("dplyr", repos = "http://cran.us.r-project.org")
library(dplyr)
install.packages("lubridate", repos = "http://cran.us.r-project.org")
library(hms)
install.packages("readr", repos = "http://cran.us.r-project.org")
library(readr)


I did some standard cleanup of data by: 

 * merging the March files with the corresponding April files, 
 * removing duplicate rows that overlapped from each file, 
 * turning a date-time character string into actual Date/Time field types for analysis (some files only included Date),
 * adding days of the week and weeks of the year which is helpful in analysis, 
 *checking for duplicate rows based based on simple primary keys (like Id, Time, Date)
 * normalizing calories, METs and HR averages per minute as it appeared they were being measured differently from dailyAcitivity metrics and minuteActivity metrics.

dailyActivity cleanup (including cleanup merging of rows with duplicate primary keys for when user entered more than 1x a day - hence the comment that pops up 'Daily Activity Needs more work, which was recified) - 

In [ ]:
bind <- rbind(read_csv("/kaggle/input/fitbit/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/dailyActivity_merged.csv", show_col_types = FALSE), read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv", show_col_types = FALSE))
bind = rename(bind, Date = ActivityDate)
bind = distinct(bind)
dateClean <- bind  %>%
  mutate(Date = mdy(Date)) 
dateClean$day_of_week <- weekdays(dateClean$Date)
dateClean$week_of_year <- week(dateClean$Date)
dailyActivity <- dateClean
notunique <- dateClean %>% 
  add_count(Id, Date) %>% 
  filter(n > 1)
if(nrow(notunique) == 0){
  glimpse(dailyActivity)
} else if (nrow(notunique) > 0) {
    print("dailyActivity needs more work")
}
dailyActivityClean <- dailyActivity %>%
  group_by(Id, Date, day_of_week, week_of_year)  %>%
  summarise(
    TotalSteps = sum(TotalSteps),
    TotalDistance  = sum(TotalDistance),  
    TrackerDistance =sum(TrackerDistance),     
    LoggedActivitiesDistance = sum(LoggedActivitiesDistance),
    VeryActiveDistance = sum(VeryActiveDistance),
    ModeratelyActiveDistance = sum(ModeratelyActiveDistance),
    LightActiveDistance   = sum(LightActiveDistance),
    SedentaryActiveDistance = sum(SedentaryActiveDistance), 
    VeryActiveMinutes  = sum(VeryActiveMinutes),
    FairlyActiveMinutes  = sum(FairlyActiveMinutes),
    LightlyActiveMinutes  = sum(LightlyActiveMinutes),
    SedentaryMinutes    = sum(SedentaryMinutes),
    Calories   = sum(Calories) , 
    .groups = 'drop'
      )
print("Resolved")
glimpse(dailyActivityClean) 

Cleaning the hourly data -

In [ ]:
bind <- rbind(read_csv("/kaggle/input/fitbit/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/hourlyCalories_merged.csv", show_col_types = FALSE), read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/hourlyCalories_merged.csv", show_col_types = FALSE))
bind = rename(bind, Date_Time = ActivityHour)
bind=distinct(bind)
clean <- bind %>%
  mutate(Date_Time = mdy_hms(Date_Time),
         Date = as.Date(Date_Time),
         Time = as_hms(Date_Time)) 
hourlyCalories <- clean
notunique <- clean %>% 
  add_count(Id, Date, Time) %>% 
  filter(n > 1)
if(nrow(notunique) == 0){
  print("hourlyCalories Results")
  glimpse(hourlyCalories) 
} else if (nrow(notunique) > 0) {
  print("hourlyCalories needs more work")
}


bind <- rbind(read_csv("/kaggle/input/fitbit/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/hourlyIntensities_merged.csv", show_col_types = FALSE), read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/hourlyIntensities_merged.csv", show_col_types = FALSE))
bind = rename(bind, Date_Time = ActivityHour)
bind = distinct(bind)
clean <- bind %>%
  mutate(Date_Time = mdy_hms(Date_Time),
         Date = as.Date(Date_Time),
         Time = as_hms(Date_Time)) 
hourlyIntensities <- clean
notunique <- clean %>% 
  add_count(Id, Date, Time) %>% 
  filter(n > 1)
if(nrow(notunique) == 0){
  print("hourlyIntensities Results")
  glimpse(hourlyIntensities) 
} else if (nrow(notunique) > 0) {
  print("hourlyIntensities needs more work")
}


bind <- rbind(read_csv("/kaggle/input/fitbit/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/hourlySteps_merged.csv", show_col_types = FALSE), read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/hourlySteps_merged.csv", show_col_types = FALSE))
bind = rename(bind, Date_Time = ActivityHour)
bind = distinct(bind)
clean <- bind %>%
  mutate(Date_Time = mdy_hms(Date_Time),
         Date = as.Date(Date_Time),
         Time = as_hms(Date_Time)) 
hourlySteps <- clean
notunique <- clean %>% 
  add_count(Id, Date, Time) %>% 
  filter(n > 1)
if(nrow(notunique) == 0){
  print("hourlySteps results")
  glimpse(hourlySteps) 
} else if (nrow(notunique) > 0) {
  print("hourlySteps needs more work")
}
hourlyStats <- merge(merge(hourlyCalories, hourlyIntensities, by.x=c("Id", "Date", "Time", "Date_Time"), by.y=c("Id", "Date", "Time", "Date_Time"), all= TRUE), hourlySteps, by.x=c("Id", "Date", "Time", "Date_Time"), by.y=c("Id", "Date", "Time", "Date_Time"), all= TRUE) 
print("hourStats cleaned and merged results")
  glimpse(hourlyStats)

Cleaning the minute data -

In [ ]:
bind <- rbind(read_csv("/kaggle/input/fitbit/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteCaloriesNarrow_merged.csv", show_col_types = FALSE), read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteCaloriesNarrow_merged.csv", show_col_types = FALSE))
bind = rename(bind, Date_Time = ActivityMinute)
bind = distinct(bind)
clean <- bind %>%
  mutate(Date_Time = mdy_hms(Date_Time),
         Date = as.Date(Date_Time),
         Time = as_hms(Date_Time)) %>%
         select (-Date_Time)
minuteCalories <- clean
notunique <- clean %>% 
  add_count(Id, Date, Time) %>% 
  filter(n > 1)
if(nrow(notunique) == 0){
  print("minuteCalories results")
  glimpse(minuteCalories)
  } else if (nrow(notunique) > 0) {
  print("minuteCalories needs more work")
}

bind <- rbind(read_csv("/kaggle/input/fitbit/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteIntensitiesNarrow_merged.csv", show_col_types = FALSE),read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteIntensitiesNarrow_merged.csv", show_col_types = FALSE) )
bind = rename(bind, Date_Time = ActivityMinute)
bind = distinct(bind)
clean <- bind %>%
  mutate(Date_Time = mdy_hms(Date_Time),
         Date = as.Date(Date_Time),
         Time = as_hms(Date_Time)) %>%
         select (-Date_Time)
minuteIntensities <- clean
notunique <- clean %>% 
  add_count(Id, Date, Time) %>% 
  filter(n > 1)
if(nrow(notunique) == 0){
  print("minuteIntensities results")
  glimpse(minuteIntensities) 
} else if (nrow(notunique) > 0) {
  print("minuteIntensities needs more work")
}

bind <- rbind(read_csv("/kaggle/input/fitbit/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteStepsNarrow_merged.csv", show_col_types = FALSE), read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteStepsNarrow_merged.csv", show_col_types = FALSE))
bind = rename(bind, Date_Time = ActivityMinute)
bind = distinct(bind)
clean <- bind %>%
  mutate(Date_Time = mdy_hms(Date_Time),
         Date = as.Date(Date_Time),
         Time = as_hms(Date_Time)) %>%
         select (-Date_Time)
minuteSteps <- clean
notunique <- clean %>% 
  add_count(Id, Date, Time) %>% 
  filter(n > 1)
if(nrow(notunique) == 0){
  print("minuteSteps results")
  glimpse(minuteSteps) 
} else if (nrow(notunique) > 0) {
  print("minuteSteps needs more work")
}
minuteStats <- merge(merge(minuteCalories, minuteIntensities, all = TRUE), minuteSteps, all = TRUE)

MinuteMETs required some special handling because of 2 bad data rows in the merged data.  NA results were inserted into METs field during a data conversion.  I used those NAs to filter out the bad data rows.

In [ ]:
bind <- rbind(read_csv("/kaggle/input/fitbit/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteMETsNarrow_merged.csv", show_col_types = FALSE), "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteMETsNarrow_merged.csv", show_col_types = FALSE)
bind = rename(bind, Date_Time = ActivityMinute)
bind = distinct(bind)
bind$METs = as.integer(bind$METs)
bind = filter(bind,!is.na(bind$METs))
clean <- bind %>%
  mutate(Date_Time = mdy_hms(Date_Time),
         Date = as.Date(Date_Time),
         Time = as_hms(Date_Time)) 
minuteMets <- clean
notunique <- clean %>% 
  add_count(Id, Date, Time) %>% 
  filter(n > 1)
if(nrow(notunique) == 0){
   print("minuteMets results")
  glimpse(minuteMets) 
} else if (nrow(notunique) > 0) {
  print("minuteMets needs more work")
}

HeartRate data came in seconds so I cleaned the data and converted it to minutes using a mean.

In [ ]:
bind <- rbind(read_csv("/kaggle/input/fitbit/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/heartrate_seconds_merged.csv", show_col_types = FALSE),read_csv("/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv", show_col_types = FALSE) )
bind = rename(bind, Date_Time = Time)
bind = distinct(bind)

clean <- bind %>%
  mutate( Date_Time = mdy_hms(Date_Time),
          Date = as.Date(Date_Time),
          Time = as_hms(Date_Time), 
          TimeInMinutes = as_hms(floor_date(Date_Time, "minute")) )
secondsHeartRate <- clean
notunique <- clean %>% 
  add_count(Id, Date, Time) %>% 
  filter(n > 1)
if(nrow(notunique) == 0){
  print("secondsHearRate results")
  glimpse(secondsHeartRate) 
} else if (nrow(notunique) > 0) {
  print("secondsHeartRate needs more work")
}
minuteHeartRate <- secondsHeartRate %>%
group_by(Id, Date, TimeInMinutes)  %>%
  summarise(
    avgMinuteHeartRate = round(mean(Value), digits=2), .groups = 'drop'
      )
minuteHeartRate = rename(minuteHeartRate, Time = TimeInMinutes)
glimpse(minuteHeartRate)

Normalizing the cleaned data -

In [ ]:
minuteStats <- merge(x=minuteStats, y= minuteMets[ , c("Id", "Date", "Time","METs")], all=TRUE)
minuteStats <- merge(x=minuteStats, y= minuteHeartRate[ , c("Id", "Date", "Time", "avgMinuteHeartRate")], all=TRUE)
perMinuteNorms <- minuteStats %>%
  group_by(Intensity) %>%
  summarise(
    AvgCalories = round(mean(Calories)),
    avgMETs = round(mean(METs,na.rm=TRUE))/10,
    avgHeartrate = round(mean(avgMinuteHeartRate, na.rm=TRUE))
     )
perMinuteNorms <- perMinuteNorms %>%
        drop_na() 
perMinuteNorms$METsTgt <- c("< 1.5", "< 3", "3-5.9", "> 6")
perMinuteNorms$HRTgt <- c("60-100 Beats Per Min", "50-60% Max HR", "60-70% Max HR", "70-85% Max HR")
perMinuteNorms <- perMinuteNorms %>%
  relocate(METsTgt, .after = avgMETs)

  
caloriesmultiplierSed = perMinuteNorms$AvgCalories 
dailyActivityClean[, "SedCalsNormd"] <- round(caloriesmultiplierSed[1] * dailyActivityClean$SedentaryMinutes)
dailyActivityClean[, "LightlyCalsNormalized"] <- round(caloriesmultiplierSed[2] * dailyActivityClean$LightlyActiveMinutes)
dailyActivityClean[, "FairlyCalsNormalized"] <- round(caloriesmultiplierSed[3] * dailyActivityClean$FairlyActiveMinutes)
dailyActivityClean[, "VeryCalsNormalized"] <- round(caloriesmultiplierSed[4] * dailyActivityClean$VeryActiveMinutes)


knitr::kable(perMinuteNorms, format="pipe", align ="c", caption="Sample METs and HR averages vs American Heart Association Targets")

This visual shows how our 35 athlete's average for METs and heartrate meet targets set by the AHA.  Max HR is 220-Age.   Age wasn't available for each of the athletes so I just included an indicator.

## Analysis

Calculate ADA Compliance to minutes and graph individual trends over time, showing who is meeting ADA guidelines and how often.   I also filtered out the very early weeks where there were few participants.

In [ ]:
weekly_data <- dailyActivityClean %>%
  mutate(mvpa_minutes = VeryActiveMinutes + FairlyActiveMinutes) %>%
  group_by(Id, week_of_year) %>%
  summarise(total_mvpa_minutes = sum(mvpa_minutes), .groups = 'drop')
  
weekly_data <- weekly_data %>%
  mutate(ada_compliant = total_mvpa_minutes >= 150) %>%
  filter(week_of_year >= 14)

color_table <- tibble(
  ada_compliant = c("TRUE", "FALSE"),
  Color = c("maroon3", "blue4")
  )


ggplot(weekly_data, aes(x = week_of_year, y = total_mvpa_minutes, fill = ada_compliant)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = color_table$Color) +
  facet_wrap(~Id) +
  ylim(-1, 750) +
  xlim(13, 20) +
  geom_hline(yintercept = 150, linetype = "dashed", color = "orange") +
  labs(       x = "Week",
       y = "Moderate-to-Vigorous Physical Activity Minutes",
       fill = "ADA Compliant") +
   ggtitle("ADA Compliance each week by Athlete") +
  theme (plot.title= element_text(hjust = 0.5), axis.text.y = element_text(size = 5))
ggsave("ADA Compliance each week by Athlete.png")


This visual gives you a quick overview of who is consistently meeting the guidelines.

In [ ]:
compliance_summary <- weekly_data %>%
  group_by(Id) %>%
  summarise(compliance_rate = mean(ada_compliant) * 100)
ggplot(compliance_summary, aes(x = reorder(as.character(Id), compliance_rate), y = compliance_rate)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  coord_flip() +
  labs(x = "Athlete ID",
       y = "Percentage of Weeks Compliant") +
  ggtitle("Percentage of Weeks ADA Compliant by Athlete") +
  theme (plot.title= element_text(hjust = 0.5))
 ggsave(file = "Percentage of Weeks ADA Compliant by Athlete.png", dpi = 600, width = 8, height = 6, units = "in")

This visual shows the percentage of athletes that met the ADA guidelines.   Less that a third of the athletes were 100% compliant across the weeks studied.

In [ ]:
ggplot(weekly_data, aes(x = as.factor(Id), y = total_mvpa_minutes)) +
  geom_boxplot() +
  geom_hline(yintercept = 150, linetype = "dashed", color = "maroon3") +
  labs(x = "Id",
       y = "Total Activity") +
  ggtitle("Distribution of Weekly Moderate-to-Vigorous Physical Activity") +
  theme(axis.text.x=element_text(angle = 90), plot.title= element_text(hjust = 0.5))
   ggsave(file = "Distribution of Weekly Moderate-to-Vigorous Physical Activity.png", dpi = 600, width = 8, height = 6, units = "in")

This visual shows each athlete's average and spread of Moderate-to-Vigorous Physical activity across the weeks relative to the AHA target.

In [ ]:
average_data <- minuteStats %>%
  group_by(Id) %>%
  summarise(
  average_heartrate = mean(avgMinuteHeartRate, na.rm=TRUE), 
  average_mets = mean(METs, na.rm=TRUE)/10, 
  average_cals = mean(Calories, na.rm=TRUE)
  )

compliance_summary_join <- left_join(compliance_summary, average_data, by = "Id")

ggplot(compliance_summary_join, aes(x = average_mets, y = average_heartrate, color = compliance_rate)) +
  geom_point() +
  labs(title = "Average Heartrate vs Average METS", x = "Average METS", y = "Average Heartrate", color = "Compliance Rate") +
  scale_color_gradient(low="maroon3", high="blue4") +
  ggtitle("Does Average Heartrate correlate to Average METs in our Sample?") +
  theme(plot.title= element_text(hjust = 0.5))
 ggsave(file = "Does Heartrate correlate to METs in our Sample.png", dpi = 600, width = 8, height = 6, units = "in")

This visual allows you to see if there is a correlation between heart rate, METs and compliance for the 14 people that tracked both Mets and heartrate.  (note the warning from the plotting tool that 21 athletes were not included because they lacked either heartrate info or METs info).  The correlation is not clear because of the very small population (14) but we should be seeing a correlation of higher heart rates to higher METs.

In [ ]:
ggplot(compliance_summary_join, aes(x = average_cals, y = average_mets, color = compliance_rate)) +
  geom_point() +
  labs(x = "Average Cals", y = "Average METs", color = "Compliance Rate") +
  scale_color_gradient(low="maroon3", high="blue") +
  ggtitle("Do METs and Calories correlate in our Sample?") +
  theme(plot.title= element_text(hjust = 0.5))
  ggsave(file = "Do METs and Calories correlate in our Sample.png", dpi = 600, width = 8, height = 6, units = "in")


This visual allows you to see if there is a correlation between METs, calories and compliance for the 34 people that tracked both calories and METs.  (note the warning from the plotting tool that 1 athlete was not included because they lacked either calorie or METs info).  But you are able to see some correlation in increased calories aligning with increased METs.

In [ ]:
ggplot(compliance_summary_join, aes(x = average_cals, y = average_heartrate, color = compliance_rate)) +
  geom_point() +
  labs( x = "Average Cals", y = "Average HR", color = "Compliance Rate") +
  scale_color_gradient(low="maroon3", high="blue") +
  ggtitle("Does Heartrate correlate to Calories Burned in our Sample?") +
  theme(plot.title= element_text(hjust = 0.5))
  ggsave(file = "Does Heartrate correlate to Calories Burned in our Sample.png", dpi = 600, width = 8, height = 6, units = "in")

This visual allows you to see if there is a correlation between heartrate and calories and compliance for the 15 people that tracked both calories and heartrate.  (note the warning from the plotting tool that 1 athlete was not included because they lacked either calorie or heartrate info).  But you are able to see some correlation in increased calories aligning with increased heartrate.

## Conclusion and Recommendation
It may be surprising to see only a third of fitbit users are actually making the exercise targets deemed necessary to live a healthy lifestyle by the American Heart Association.  I would consider fitbit users more conscious of the need for movement so the number could actually be lower across the non-fitbit population.

I believe it would be most beneficial to understand why people are not meeting guidelines (both fitbit users and general population).   Is it lack of motivation, perceived barriers, insufficient knowledge?  Polling people to understand what's holding them back despite AHA guidelines could identify key marketing messages for tracking exercise.    

Some ideas on roadblocks - 

 * do non-fitbit users know how much exercise they're currently getting to even determine how they do against ADA targets?   BellaBeat tools are a great way to start!
 
 * do people feel it's too hard to meet targets?  Does BellaBeat need to communicate the AHA targets aim to improve well-being and aren't about elite fitness?
 
 * can BellaBeat leverage the scientific studies completed by the ADA to point out the value of healthy longevity?
 
 * can BellaBeat show, using their tools, a real return on investment between working toward ADA targets the athlete's metrics of improved sleep, increased energy, weight loss.
 
Polling needs to be completed to identify roadblocks for people.   
A/B tests need to be conducted to see how different messages for handling of these roadblocks, as well as incentives, are received by potential customers.  

By shifting the focus from intense fitness to accessible well-being, addressing perceived barriers, and providing targeted support, BellaBeat can reach the majority of users who are not currently meeting ADA guidelines as a target audience for their offerings.


